In [1]:
import click
import json
import numpy as np
import pyvista as pv
import scipy.spatial as ss
from pymeshfix import MeshFix
import pandas as pd
import math
import matplotlib.pyplot as plt
from tqdm import tqdm
from shapely.geometry import MultiPolygon, Polygon
import mapbox_earcut as earcut

In [2]:
filename = "/Users/liberostelios/Dropbox/CityJSON/Helsinki/CityGML_BUILDINGS_LOD2_NOTEXTURES_672496x2.json"

with open(filename) as file:
    cm = json.load(file)

if "transform" in cm:
    s = cm["transform"]["scale"]
    t = cm["transform"]["translate"]
    verts = [[v[0] * s[0] + t[0], v[1] * s[1] + t[1], v[2] * s[2] + t[2]]
            for v in cm["vertices"]]
else:
    verts = cm["vertices"]

# mesh points
vertices = np.array(verts)

In [3]:
def get_surface_boundaries(geom):
    """Returns the boundaries for all surfaces"""

    if geom["type"] == "MultiSurface":
        return geom["boundaries"]
    elif geom["type"] == "Solid":
        return geom["boundaries"][0]
    else:
        raise Exception("Geometry not supported")

def to_polydata(geom, vertices):
    """Returns the polydata mesh from a CityJSON geometry"""

    boundaries = get_surface_boundaries(geom)

    f = [[len(r[0])] + r[0] for r in [f for f in boundaries]]
    faces = np.hstack(f)
    print(vertices[f[9][1:]])

    mesh = pv.PolyData(vertices, faces, n_faces=len(boundaries))

    if "semantics" in geom:        
        semantics = geom["semantics"]
        if geom["type"] == "MultiSurface":
            values = semantics["values"]
        else:
            values = semantics["values"][0]
        
        mesh["semantics"] = [semantics["surfaces"][i]["type"] for i in values]
    
    return mesh

In [4]:
def project_2d(points, normal):
    origin = points[0]
    
    if normal[2] > 0.001 or normal[2] < -0.001:
        x_axis = [1, 0, -normal[0]/normal[2]];
    elif normal[1] > 0.001 or normal[1] < -0.001:
        x_axis = [1, -normal[0]/normal[1], 0];
    else:
        x_axis = [-normal[1] / normal[0], 1, 0];
    
    y_axis = np.cross(normal, x_axis)
    
    return[]
     
#     return [[np.dot(p - origin, x_axis), np.dot(p - origin, y_axis)] for p in points]

def triangulate(mesh):
    """Triangulates a mesh in the proper way"""
    
    final_mesh = pv.PolyData()
    for i in np.arange(mesh.n_cells):
        p = project_2d(mesh.cell_points(i), mesh.face_normals[i])
        result = earcut.triangulate_float32(p, [len(p)])

        triangles = np.hstack([[3] + list(t) for t in result.reshape(-1,3)])
        
        new_mesh = pv.PolyData(mesh.cell_points(i), triangles)
        for k in mesh.cell_arrays:
            new_mesh[k] = [mesh.cell_arrays[k][i] for _ in np.arange(len(triangles)/4)]
        
        final_mesh = final_mesh + new_mesh
    
    return final_mesh

In [5]:
# obj = "GUID_816CA7F9-6357-447D-96E3-C74C5E47AABF_2"
obj = next(iter(cm["CityObjects"]))

building = cm["CityObjects"][obj]

In [6]:
dataset = to_polydata(building["geometry"][1], vertices)
dataset = dataset.clean()
dataset

[[2.54971636e+07 6.67288377e+06 2.73300000e+01]
 [2.54971636e+07 6.67288377e+06 2.51100000e+01]
 [2.54971632e+07 6.67289170e+06 2.40170000e+01]
 [2.54971632e+07 6.67289170e+06 3.08000000e+00]
 [2.54971639e+07 6.67287695e+06 3.08000000e+00]
 [2.54971639e+07 6.67287695e+06 2.73300000e+01]]


PolyData (0x1420618e0)
  N Cells:	23
  N Points:	40
  X Bounds:	2.550e+07, 2.550e+07
  Y Bounds:	6.673e+06, 6.673e+06
  Z Bounds:	3.080e+00, 2.733e+01
  N Arrays:	1

In [7]:
trimesh = triangulate(dataset).clean()
trimesh.plot(show_edges=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [8]:
side = dataset.extract_cells([9])#.plot(show_edges=True, color='grey')

pts = side.points
# idx = [6, 1, 0, 4, 3, 2, 5]
idx = [6, 5, 2, 3, 4, 0, 1]

sface = pv.PolyData(pts, idx, n_faces=1)
sface.clean().plot(show_edges=True)

# side.points[[1, 0, 4, 3, 2, 5]]

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [11]:
def project_2d(points, normal):
    origin = points[0]
    
    if normal[2] > 0.001 or normal[2] < -0.001:
        x_axis = [1, 0, -normal[0]/normal[2]];
    elif normal[1] > 0.001 or normal[1] < -0.001:
        x_axis = [1, -normal[0]/normal[1], 0];
    else:
        x_axis = [-normal[1] / normal[0], 1, 0];
    
    y_axis = np.cross(normal, x_axis)
     
    return [[np.dot(p - origin, x_axis), np.dot(p - origin, y_axis)] for p in points]

In [12]:
project_2d(pts, sface.face_normals[0])

[[0.0, 0.0],
 [0.0, -44.665332018021125],
 [137.30414922532034, 443.2330019626153],
 [-159.80587571488664, 443.2330019626153],
 [-159.80587571488664, 21.99063418725095],
 [137.30414922532034, -44.665332018021125]]

In [13]:
edges = trimesh.extract_feature_edges(boundary_edges=True,
                           feature_edges=False,
                           manifold_edges=False)

p = pv.Plotter()

p.add_mesh(trimesh)
p.add_mesh(dataset.extract_cells([9]).triangulate())
if trimesh.n_open_edges:
    p.add_mesh(edges, color='black')

p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [14]:
clean = trimesh.clean()
voxel = pv.voxelize(clean, density=clean.length/100, check_surface=False)
voxel.plot(show_edges=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [15]:
print(f"Voxel: {voxel.volume}")
print(f"Actual: {trimesh.volume}")

Voxel: 12216.08386914232
Actual: 11631.29785567844


In [16]:
p = pv.Plotter()

p.add_mesh(voxel, opacity=0.2, show_edges=True)
p.add_mesh(pv.PolyData(np.mean(voxel.cell_centers().points, axis=0)), color='white')

p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [17]:
from helpers.minimumBoundingBox import MinimumBoundingBox

In [18]:
obb_2d = MinimumBoundingBox([(p[0], p[1]) for p in dataset.clean().points])
obb_2d.area

604.9247202999668

In [19]:
obb_2d.area

604.9247202999668

In [20]:
ground_z = np.min(dataset.clean().points[:, 2])
height = np.max(dataset.clean().points[:, 2]) - ground_z
box = np.array([[p[0], p[1], ground_z] for p in list(obb_2d.corner_points)])

In [21]:
obb = pv.PolyData(box).delaunay_2d()
pts = obb.points

t = np.mean(pts, axis=0)

obb.points = obb.points - t
obb = obb.extrude([0.0, 0.0, height])
obb.points = obb.points + t

In [30]:
p = pv.Plotter()

p.add_mesh(obb, opacity=0.3)
p.add_mesh(trimesh)

p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [25]:
dataset.clean().triangulate().volume

178300040.24164733

In [26]:
m = MeshFix(obb.clean().triangulate())
m.repair()
fixed_obb = m.mesh

In [27]:
pv.PolyData(box).delaunay_2d().area * height

14669.424465904782

In [28]:
obb.faces

array([3, 3, 2, 1, 3, 7, 6, 5, 3, 3, 1, 0, 3, 7, 5, 4])

In [29]:
from pyobb.obb import OBB

obb_full_3d = OBB.build_from_points(dataset.clean().points)
obb_full_3d.points

[array([25497178.057, 6672901.955, 22.019]),
 array([25497157.657, 6672897.673, 21.771]),
 array([25497165.379, 6672860.054, 36.077]),
 array([25497185.779, 6672864.335, 36.325]),
 array([25497160.392, 6672886.438, -9.251]),
 array([25497180.792, 6672890.719, -9.003]),
 array([25497188.514, 6672853.100, 5.302]),
 array([25497168.114, 6672848.818, 5.054])]